# HMM for FS4_new (Handpicked OF/OP + P + GazeDist)

In [1]:
# Imports
import pandas as pd
import numpy as np
from pandas import *
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.metrics import accuracy_score
import random

In [2]:
# Load Data
file3 = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/FS4_new/p3_data_FS4new.csv'
file5 = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/FS4_new/p5_data_FS4new.csv'
file7 = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/FS4_new/p7_data_FS4new.csv'
file8 = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/FS4_new/p8_data_FS4new.csv'
file9 = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/FS4_new/p9_data_FS4new.csv'
file10 = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/FS4_new/p10_data_FS4new.csv'

data3 = pd.read_csv(file3)
data5 = pd.read_csv(file5)
data7 = pd.read_csv(file7)
data8 = pd.read_csv(file8)
data9 = pd.read_csv(file9)
data10 = pd.read_csv(file10)

Note: keep timestamp and session_num until right before running the model in all feature sets

### Step 1: Train Test Split 

Train on Earlier Sessions and Test on Later Sessions

In [121]:
# Feature Set 4_new is used for each participant in set
FS = data10
FS = FS.sort_values(['session_num', 'timestamp'])

In [4]:
def find_threshold(FS, size):
    sessions = FS['session_num'].unique()
    thresh = size*(len(sessions))
    print(sessions)
    thresh = round(thresh,0)
    return sessions[int(thresh) - 1]

In [5]:
def session_split(sess_threshold):
    print(sess_threshold)
    train = FS[(FS['session_num'] <= sess_threshold)] 
    test = FS[(FS['session_num'] > sess_threshold)]

    # drop columns needed for split
    train2 = train.drop(columns=['timestamp'])
    test2 = test.drop(columns=['timestamp'])

    X_train2 = train2.drop(columns=['engagement'])
    y_train2 = train2['engagement']

    X_test2 = test2.drop(columns=['engagement'])
    y_test2 = test2['engagement']
    
    return X_train2, y_train2, X_test2, y_test2

In [127]:
# choose train-test split
#X_train, y_train, X_test, y_test = session_split(find_threshold(FS,0.7))
#X_train, y_train, X_test, y_test = session_split(find_threshold(FS,0.5))
X_train, y_train, X_test, y_test = session_split(find_threshold(FS,0.3))

[ 2.1  2.2  3.1  3.2  4.2  4.3  4.4  5.1  5.2  7.1  7.2  8.   9. ]
3.2


In [128]:
# make HMM train-test inputs
lengths_train = X_train['session_num'].value_counts(sort=False).tolist()
X_train = X_train.drop(columns=['session_num'])
lengths_test = X_test['session_num'].value_counts(sort=False).tolist()
X_test = X_test.drop(columns=['session_num'])

### Step 2: Implement the HMM

In [8]:
from hmmlearn import hmm
from hmmlearn.hmm import GaussianHMM
# 3 questions from HMM:
# (1) given model, what is likelihood of sequence S happening?
# (2) given model and sequence S, what is optimal hidden state sequence?
# (3) given sequence S and # of hidden states, what is optimal model to maximize probability of S?

In [129]:
# HMM tends to get stuck in local minimum. For best results, run several times and pick best
best_model = GaussianHMM(n_components=2, n_iter=5, algorithm = 'viterbi').fit(X_train, lengths_train)
best_pred = best_model.predict(X_train, lengths_train)
best = accuracy_score(y_train, best_pred)
num = 0

for i in range(0,10):
    new_model = GaussianHMM(n_components=2, n_iter=5, algorithm = 'viterbi', init_params="mcs")
    
    # randomly initialize probability matrix 
    a = random.uniform(0, 1)
    b = random.uniform(0, 1)
    new_model.transmat_ = np.array([[a, 1-a], [b, 1-b]])
    new_model = new_model.fit(X_train, lengths_train)
    
    # evaluate this model and see if better than best so far 
    new_pred = new_model.predict(X_train, lengths_train)
    new_acc = accuracy_score(y_train, new_pred)
    print(i, new_acc)
    
    if (new_acc > best):
        best = new_acc
        best_model = new_model
        num = i
         
hmm_model = best_model

0 0.30283795159
1 0.453608424525
2 0.595076531927
3 0.697462678875
4 0.698072529247
5 0.301575303637
6 0.546391575475
7 0.404923468073
8 0.595076531927
9 0.613372043085


# 70-30 Splits

### P3 FS4_new 70-30

In [10]:
# model takes array of dimension (n_samples, n_features)
model_p3 = hmm_model
print(model_p3.monitor_.history)

deque([1484808.2228910511, 1484808.2228911791], maxlen=2)


In [11]:
#evaluate FS4_new P3 HMM
FS4new_pred3 = model_p3.predict(X_test, lengths_test)
scores = model_p3.predict_proba(X_test.values, lengths_test)
scores = scores[: ,1]

print("Accuracy for FS4_new P3 HMM:",accuracy_score(y_test, FS4new_pred3))
print("AUC:",roc_auc_score(y_test, scores)) 
print(metrics.confusion_matrix(y_test, FS4new_pred3))
print(metrics.classification_report(y_test, FS4new_pred3))

Accuracy for FS4_new P3 HMM: 0.71677864875
AUC: 0.7386119225
[[ 7173  2118]
 [13057 31232]]
             precision    recall  f1-score   support

        0.0       0.35      0.77      0.49      9291
        1.0       0.94      0.71      0.80     44289

avg / total       0.84      0.72      0.75     53580



### P5 FS4_new 70-30

In [16]:
# model takes array of dimension (n_samples, n_features)
model_p5 = hmm_model
print(model_p5.monitor_.history)

deque([12282441.888420505, 12282441.888408747], maxlen=2)


In [17]:
#evaluate FS4_new P5 HMM
FS4new_pred5 = model_p5.predict(X_test, lengths_test)
scores = model_p5.predict_proba(X_test.values, lengths_test)
scores = scores[: ,1]

print("Accuracy for FS4_new P5 HMM:",accuracy_score(y_test, FS4new_pred5))
print("AUC:",roc_auc_score(y_test, scores)) 
print(metrics.confusion_matrix(y_test, FS4new_pred5))
print(metrics.classification_report(y_test, FS4new_pred5))

Accuracy for FS4_new P5 HMM: 0.700621308583
AUC: 0.695488714772
[[ 6249  2839]
 [12773 30287]]
             precision    recall  f1-score   support

          0       0.33      0.69      0.44      9088
          1       0.91      0.70      0.80     43060

avg / total       0.81      0.70      0.73     52148



### P7 FS4_new 70-30

In [23]:
# model takes array of dimension (n_samples, n_features)
model_p7 = hmm_model
print(model_p7.monitor_.history)

deque([8390809.2739644442, 8390809.273962304], maxlen=2)


In [24]:
#evaluate FS4_new P7 HMM
FS4new_pred7 = model_p7.predict(X_test, lengths_test)
scores = model_p7.predict_proba(X_test.values, lengths_test)
scores = scores[: ,1]

print("Accuracy for FS4_new P7 HMM:",accuracy_score(y_test, FS4new_pred7))
print("AUC:",roc_auc_score(y_test, scores)) 
print(metrics.confusion_matrix(y_test, FS4new_pred7))
print(metrics.classification_report(y_test, FS4new_pred7))

Accuracy for FS4_new P7 HMM: 0.833556401819
AUC: 0.811524241634
[[75036  1217]
 [21292 37690]]
             precision    recall  f1-score   support

        0.0       0.78      0.98      0.87     76253
        1.0       0.97      0.64      0.77     58982

avg / total       0.86      0.83      0.83    135235



### P8 FS4_new 70-30

In [35]:
# model takes array of dimension (n_samples, n_features)
model_p8 = hmm_model
print(model_p8.monitor_.history)

deque([2744713.4677161686, 2773171.3350919969], maxlen=2)


In [36]:
#evaluate FS4_new P8 HMM
FS4new_pred8 = model_p8.predict(X_test, lengths_test)
scores = model_p8.predict_proba(X_test.values, lengths_test)
scores = scores[: ,1]

print("Accuracy for FS4_new P8 HMM:",accuracy_score(y_test, FS4new_pred8))
print("AUC:",roc_auc_score(y_test, scores)) 
print(metrics.confusion_matrix(y_test, FS4new_pred8))
print(metrics.classification_report(y_test, FS4new_pred8))

Accuracy for FS4_new P8 HMM: 0.334839949758
AUC: 0.376723164335
[[24198 40379]
 [29524 10991]]
             precision    recall  f1-score   support

        0.0       0.45      0.37      0.41     64577
        1.0       0.21      0.27      0.24     40515

avg / total       0.36      0.33      0.34    105092



### P9 FS4_new 70-30

In [41]:
# model takes array of dimension (n_samples, n_features)
model_p9 = hmm_model
print(model_p9.monitor_.history)

deque([5919137.5161923338, 8910856.4610294867], maxlen=2)


In [42]:
#evaluate FS4_new P9 HMM
FS4new_pred9 = model_p9.predict(X_test, lengths_test)
scores = model_p9.predict_proba(X_test.values, lengths_test)
scores = scores[: ,1]

print("Accuracy for FS4_new P9 HMM:",accuracy_score(y_test, FS4new_pred9))
print("AUC:",roc_auc_score(y_test, scores)) 
print(metrics.confusion_matrix(y_test, FS4new_pred9))
print(metrics.classification_report(y_test, FS4new_pred9))

Accuracy for FS4_new P9 HMM: 0.481946787658
AUC: 0.551341538457
[[35052 72188]
 [22695 53218]]
             precision    recall  f1-score   support

          0       0.61      0.33      0.42    107240
          1       0.42      0.70      0.53     75913

avg / total       0.53      0.48      0.47    183153



### P10 FS4_new 70-30

In [47]:
# model takes array of dimension (n_samples, n_features)
model_p10 = hmm_model
print(model_p10.monitor_.history)

deque([5922755.8721404951, 5923405.8099162905], maxlen=2)


In [48]:
#evaluate FS4_new P10 HMM
FS4new_pred10 = model_p10.predict(X_test, lengths_test)
scores = model_p10.predict_proba(X_test.values, lengths_test)
scores = scores[: ,1]

print("Accuracy for FS4_new P10 HMM:",accuracy_score(y_test, FS4new_pred10))
print("AUC:",roc_auc_score(y_test, scores)) 
print(metrics.confusion_matrix(y_test, FS4new_pred10))
print(metrics.classification_report(y_test, FS4new_pred10))

Accuracy for FS4_new P10 HMM: 0.343454908759
AUC: 0.37343898173
[[ 91832 236836]
 [ 41136  53582]]
             precision    recall  f1-score   support

          0       0.69      0.28      0.40    328668
          1       0.18      0.57      0.28     94718

avg / total       0.58      0.34      0.37    423386



# 50-50 Splits

### P3 FS4_new 50-50

In [55]:
# model takes array of dimension (n_samples, n_features)
model_p3 = hmm_model
print(model_p3.monitor_.history)

deque([1145793.8861379013, 1145793.8861376292], maxlen=2)


In [56]:
#evaluate FS4_new P3 HMM
FS4new_pred3 = model_p3.predict(X_test, lengths_test)
scores = model_p3.predict_proba(X_test.values, lengths_test)
scores = scores[: ,1]

print("Accuracy for FS4_new P3 HMM:",accuracy_score(y_test, FS4new_pred3))
print("AUC:",roc_auc_score(y_test, scores)) 
print(metrics.confusion_matrix(y_test, FS4new_pred3))
print(metrics.classification_report(y_test, FS4new_pred3))

Accuracy for FS4_new P3 HMM: 0.716374911181
AUC: 0.727538936806
[[10241  3508]
 [18446 45210]]
             precision    recall  f1-score   support

        0.0       0.36      0.74      0.48     13749
        1.0       0.93      0.71      0.80     63656

avg / total       0.83      0.72      0.75     77405



### P5 FS4_new 50-50

In [68]:
# model takes array of dimension (n_samples, n_features)
model_p5 = hmm_model
print(model_p5.monitor_.history)

deque([11188393.266187027, 11188393.266199561], maxlen=2)


In [69]:
#evaluate FS4_new P5 HMM
FS4new_pred5 = model_p5.predict(X_test, lengths_test)
scores = model_p5.predict_proba(X_test.values, lengths_test)
scores = scores[: ,1]

print("Accuracy for FS4_new P5 HMM:",accuracy_score(y_test, FS4new_pred5))
print("AUC:",roc_auc_score(y_test, scores)) 
print(metrics.confusion_matrix(y_test, FS4new_pred5))
print(metrics.classification_report(y_test, FS4new_pred5))

Accuracy for FS4_new P5 HMM: 0.776544927075
AUC: 0.696560032543
[[13257 10017]
 [18097 84444]]
             precision    recall  f1-score   support

          0       0.42      0.57      0.49     23274
          1       0.89      0.82      0.86    102541

avg / total       0.81      0.78      0.79    125815



### P7 FS4_new 50-50

In [80]:
# model takes array of dimension (n_samples, n_features)
model_p7 = hmm_model
print(model_p7.monitor_.history)

deque([8174151.6034516497, 8174151.6034622965], maxlen=2)


In [81]:
#evaluate FS4_new P7 HMM
FS4new_pred7 = model_p7.predict(X_test, lengths_test)
scores = model_p7.predict_proba(X_test.values, lengths_test)
scores = scores[: ,1]

print("Accuracy for FS4_new P7 HMM:",accuracy_score(y_test, FS4new_pred7))
print("AUC:",roc_auc_score(y_test, scores)) 
print(metrics.confusion_matrix(y_test, FS4new_pred7))
print(metrics.classification_report(y_test, FS4new_pred7))

Accuracy for FS4_new P7 HMM: 0.779357201108
AUC: 0.790773740645
[[83249  3195]
 [37659 61056]]
             precision    recall  f1-score   support

        0.0       0.69      0.96      0.80     86444
        1.0       0.95      0.62      0.75     98715

avg / total       0.83      0.78      0.77    185159



### P8 FS4_new 50-50

In [103]:
# model takes array of dimension (n_samples, n_features)
model_p8 = hmm_model
print(model_p8.monitor_.history)

deque([2005965.8121410313, 2018221.8680601474], maxlen=2)


In [104]:
#evaluate FS4_new P8 HMM
FS4new_pred8 = model_p8.predict(X_test, lengths_test)
scores = model_p8.predict_proba(X_test.values, lengths_test)
scores = scores[: ,1]

print("Accuracy for FS4_new P8 HMM:",accuracy_score(y_test, FS4new_pred8))
print("AUC:",roc_auc_score(y_test, scores)) 
print(metrics.confusion_matrix(y_test, FS4new_pred8))
print(metrics.classification_report(y_test, FS4new_pred8))

Accuracy for FS4_new P8 HMM: 0.440703663704
AUC: 0.520482743825
[[12088 73063]
 [ 7724 51569]]
             precision    recall  f1-score   support

        0.0       0.61      0.14      0.23     85151
        1.0       0.41      0.87      0.56     59293

avg / total       0.53      0.44      0.37    144444



### P9 FS4_new 50-50

In [119]:
# model takes array of dimension (n_samples, n_features)
model_p9 = hmm_model
print(model_p9.monitor_.history)

deque([6929385.1297414964, 10065307.728199605], maxlen=2)


In [120]:
#evaluate FS4_new P9 HMM
FS4new_pred9 = model_p9.predict(X_test, lengths_test)
scores = model_p9.predict_proba(X_test.values, lengths_test)
scores = scores[: ,1]

print("Accuracy for FS4_new P9 HMM:",accuracy_score(y_test, FS4new_pred9))
print("AUC:",roc_auc_score(y_test, scores)) 
print(metrics.confusion_matrix(y_test, FS4new_pred9))
print(metrics.classification_report(y_test, FS4new_pred9))

Accuracy for FS4_new P9 HMM: 0.508636900118
AUC: 0.575512449616
[[41652 74600]
 [23651 60053]]
             precision    recall  f1-score   support

          0       0.64      0.36      0.46    116252
          1       0.45      0.72      0.55     83704

avg / total       0.56      0.51      0.50    199956



### P10 FS4_new 50-50

In [125]:
# model takes array of dimension (n_samples, n_features)
model_p10 = hmm_model
print(model_p10.monitor_.history)

deque([7048466.6991112325, 7503006.7531883093], maxlen=2)


In [126]:
#evaluate FS4_new P10 HMM
FS4new_pred10 = model_p10.predict(X_test, lengths_test)
scores = model_p10.predict_proba(X_test.values, lengths_test)
scores = scores[: ,1]

print("Accuracy for FS4_new P10 HMM:",accuracy_score(y_test, FS4new_pred10))
print("AUC:",roc_auc_score(y_test, scores)) 
print(metrics.confusion_matrix(y_test, FS4new_pred10))
print(metrics.classification_report(y_test, FS4new_pred10))

Accuracy for FS4_new P10 HMM: 0.717349829658
AUC: 0.700121683563
[[269950  82190]
 [ 75444 130116]]
             precision    recall  f1-score   support

          0       0.78      0.77      0.77    352140
          1       0.61      0.63      0.62    205560

avg / total       0.72      0.72      0.72    557700



# 30-70 Splits

### P3 FS4_new 30-70

In [60]:
# model takes array of dimension (n_samples, n_features)
model_p3 = hmm_model
print(model_p3.monitor_.history)

deque([621787.28552293032, 718104.01118034741], maxlen=2)


In [61]:
#evaluate FS4_new P3 HMM
FS4new_pred3 = model_p3.predict(X_test, lengths_test)
scores = model_p3.predict_proba(X_test.values, lengths_test)
scores = scores[: ,1]

print("Accuracy for FS4_new P3 HMM:",accuracy_score(y_test, FS4new_pred3))
print("AUC:",roc_auc_score(y_test, scores)) 
print(metrics.confusion_matrix(y_test, FS4new_pred3))
print(metrics.classification_report(y_test, FS4new_pred3))

Accuracy for FS4_new P3 HMM: 0.747819229253
AUC: 0.743014393699
[[12463  4473]
 [21748 65293]]
             precision    recall  f1-score   support

        0.0       0.36      0.74      0.49     16936
        1.0       0.94      0.75      0.83     87041

avg / total       0.84      0.75      0.78    103977



### P5 FS4_new 30-70

In [73]:
# model takes array of dimension (n_samples, n_features)
model_p5 = hmm_model
print(model_p5.monitor_.history)

deque([2148063.9927106481, 2275317.3949539112], maxlen=2)


In [74]:
#evaluate FS4_new P5 HMM
FS4new_pred5 = model_p5.predict(X_test, lengths_test)
scores = model_p5.predict_proba(X_test.values, lengths_test)
scores = scores[: ,1]

print("Accuracy for FS4_new P5 HMM:",accuracy_score(y_test, FS4new_pred5))
print("AUC:",roc_auc_score(y_test, scores)) 
print(metrics.confusion_matrix(y_test, FS4new_pred5))
print(metrics.classification_report(y_test, FS4new_pred5))

Accuracy for FS4_new P5 HMM: 0.751096189021
AUC: 0.785596487911
[[ 85209  13803]
 [ 55735 124630]]
             precision    recall  f1-score   support

          0       0.60      0.86      0.71     99012
          1       0.90      0.69      0.78    180365

avg / total       0.80      0.75      0.76    279377



### P7 FS4_new 30-70

In [85]:
# model takes array of dimension (n_samples, n_features)
model_p7 = hmm_model
print(model_p7.monitor_.history)

deque([5481868.1197526753, 5481868.1197468825], maxlen=2)


In [86]:
#evaluate FS4_new P7 HMM
FS4new_pred7 = model_p7.predict(X_test, lengths_test)
scores = model_p7.predict_proba(X_test.values, lengths_test)
scores = scores[: ,1]

print("Accuracy for FS4_new P7 HMM:",accuracy_score(y_test, FS4new_pred7))
print("AUC:",roc_auc_score(y_test, scores)) 
print(metrics.confusion_matrix(y_test, FS4new_pred7))
print(metrics.classification_report(y_test, FS4new_pred7))

Accuracy for FS4_new P7 HMM: 0.745172475752
AUC: 0.762336701353
[[95689 14278]
 [52378 99228]]
             precision    recall  f1-score   support

        0.0       0.65      0.87      0.74    109967
        1.0       0.87      0.65      0.75    151606

avg / total       0.78      0.75      0.75    261573



### P8 FS4_new 30-70

In [108]:
# model takes array of dimension (n_samples, n_features)
model_p8 = hmm_model
print(model_p8.monitor_.history)

deque([1887568.3394431826, 1908349.6150034701], maxlen=2)


In [109]:
#evaluate FS4_new P8 HMM
FS4new_pred8 = model_p8.predict(X_test, lengths_test)
scores = model_p8.predict_proba(X_test.values, lengths_test)
scores = scores[: ,1]

print("Accuracy for FS4_new P8 HMM:",accuracy_score(y_test, FS4new_pred8))
print("AUC:",roc_auc_score(y_test, scores)) 
print(metrics.confusion_matrix(y_test, FS4new_pred8))
print(metrics.classification_report(y_test, FS4new_pred8))

Accuracy for FS4_new P8 HMM: 0.486877869247
AUC: 0.560463683463
[[20536 70664]
 [ 8247 54339]]
             precision    recall  f1-score   support

        0.0       0.71      0.23      0.34     91200
        1.0       0.43      0.87      0.58     62586

avg / total       0.60      0.49      0.44    153786



### P9 FS4_new 30-70

In [114]:
# model takes array of dimension (n_samples, n_features)
model_p9 = hmm_model
print(model_p9.monitor_.history)

deque([3786942.3555388898, 4094171.5969754853], maxlen=2)


In [115]:
#evaluate FS4_new P9 HMM
FS4new_pred9 = model_p9.predict(X_test, lengths_test)
scores = model_p9.predict_proba(X_test.values, lengths_test)
scores = scores[: ,1]

print("Accuracy for FS4_new P9 HMM:",accuracy_score(y_test, FS4new_pred9))
print("AUC:",roc_auc_score(y_test, scores)) 
print(metrics.confusion_matrix(y_test, FS4new_pred9))
print(metrics.classification_report(y_test, FS4new_pred9))

Accuracy for FS4_new P9 HMM: 0.49095137572
AUC: 0.486581194268
[[81946 75034]
 [67240 55270]]
             precision    recall  f1-score   support

          0       0.55      0.52      0.54    156980
          1       0.42      0.45      0.44    122510

avg / total       0.49      0.49      0.49    279490



### P10 FS4_new 30-70

In [130]:
# model takes array of dimension (n_samples, n_features)
model_p10 = hmm_model
print(model_p10.monitor_.history)

deque([5581996.1340877358, 6006488.4181838408], maxlen=2)


In [131]:
#evaluate FS4_new P10 HMM
FS4new_pred10 = model_p10.predict(X_test, lengths_test)
scores = model_p10.predict_proba(X_test.values, lengths_test)
scores = scores[: ,1]

print("Accuracy for FS4_new P10 HMM:",accuracy_score(y_test, FS4new_pred10))
print("AUC:",roc_auc_score(y_test, scores)) 
print(metrics.confusion_matrix(y_test, FS4new_pred10))
print(metrics.classification_report(y_test, FS4new_pred10))

Accuracy for FS4_new P10 HMM: 0.498192818294
AUC: 0.480990905016
[[185962 179566]
 [140592 131890]]
             precision    recall  f1-score   support

          0       0.57      0.51      0.54    365528
          1       0.42      0.48      0.45    272482

avg / total       0.51      0.50      0.50    638010

